<a href="https://colab.research.google.com/github/Anjasfedo/eceg-lsb-lzw-huffman/blob/main/LSB/lsb_xor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import random
from sympy import isprime, randprime
import math

# Function to generate a prime number
def generate_prime(bits=512):
    return randprime(2**(bits-1), 2**bits)

# Function to compute the greatest common divisor (GCD)
def gcd(a, b):
    while b:
        a, b = b, a % b
    return a

# Function to compute modular inverse
def mod_inverse(e, phi):
    g, x, y = extended_gcd(e, phi)
    if g != 1:
        raise ValueError("Modular inverse does not exist")
    else:
        return x % phi

# Extended Euclidean Algorithm
def extended_gcd(a, b):
    if a == 0:
        return b, 0, 1
    g, x1, y1 = extended_gcd(b % a, a)
    x = y1 - (b // a) * x1
    y = x1
    return g, x, y

# Function to generate RSA keys
def generate_rsa_keys(bits=512):
    p = generate_prime(bits)
    q = generate_prime(bits)
    n = p * q
    phi = (p - 1) * (q - 1)

    # Choose e such that 1 < e < phi and gcd(e, phi) = 1
    e = random.randint(2, phi - 1)
    while gcd(e, phi) != 1:
        e = random.randint(2, phi - 1)

    d = mod_inverse(e, phi)

    public_key = (e, n)
    private_key = (d, n)

    return public_key, private_key

# Encryption function
def encrypt(message, public_key):
    e, n = public_key
    encrypted_message = [pow(ord(char), e, n) for char in message]
    return encrypted_message

# Decryption function
def decrypt(encrypted_message, private_key):
    d, n = private_key
    decrypted_message = ''.join([chr(pow(char, d, n)) for char in encrypted_message])
    return decrypted_message

# Run RSA key generation
public_key, private_key = generate_rsa_keys()

# Display keys
print("Public Key:", public_key)
print("Private Key:", private_key)

# Encrypt and decrypt a message
message = "Hello, RSA in Google Colab!"
encrypted_msg = encrypt(message, public_key)
decrypted_msg = decrypt(encrypted_msg, private_key)

print("\nOriginal Message:", message)
print("Encrypted Message:", encrypted_msg)
print("Decrypted Message:", decrypted_msg)


Public Key: (43159462927967341475307094092231448130955917556033772235359918997602431715724025508111017117208380900325732735361127175322832103877548175940155650679352335606758197409759161640059338655045678654109782895894546977645876105903960096030424214313191803526953905777700976552882989646376512045883369638825438841749, 126441065316544532985655114943356829302708133825108607594613272433259704843379755999856685994482740561147184394242311390843601559885032141682300112457999348790207875166105663644086432013636225937789939393600206907805879756163569297503436507475707701216103241570546494519232273490606349297592105242889885432469)
Private Key: (28952542248985875417214570214971205041583968757119957299666838869491256678412913444653526569916399501716377819945180192428155170109382892335919006077928012780336179290382758686703876585280847966383495891538751476982282629625782216356285084000254641716643920950723762876865011426779140135069747375217740623001, 126441065316544532985655114943356829302708

In [30]:
import random

class Point:
    def __init__(self, x=None, y=None):
        self.x = x
        self.y = y

    def is_infinity(self):
        return self.x is None and self.y is None

    def __eq__(self, other):
        if isinstance(other, Point):
            return self.x == other.x and self.y == other.y
        return False

    def __hash__(self):
        return hash((self.x, self.y))

    def __repr__(self):
        if self.is_infinity():
            return "Point at Infinity"
        return f"Point({self.x}, {self.y})"

class EllipticCurveElGamal:
    def __init__(self):
        self.a = 214
        self.b = 110
        self.p = 233
        self.base_point = self.generate_random_valid_point()

        self.characters = [chr(i) for i in range(1, 256)]
        self.valid_points = self.get_all_points()
        self.point_to_char, self.char_to_point = self.create_mappings()

    def elliptic_curve_equation(self, x):
        return (x**3 + self.a*x + self.b) % self.p

    def is_on_curve(self, x, y):
        return (y**2 - (x**3 + self.a * x + self.b)) % self.p == 0

    def generate_random_valid_point(self):
        while True:
            x = random.randint(0, self.p - 1)
            y_squared = (x**3 + self.a * x + self.b) % self.p
            if pow(y_squared, (self.p - 1) // 2, self.p) == 1:
                for y in range(self.p):
                    if (y**2) % self.p == y_squared:
                        return Point(x, y)

    def calc_point_add(self, P, Q):
        if P.is_infinity():
            return Q
        if Q.is_infinity():
            return P

        if P.x == Q.x and (P.y != Q.y or P.y == 0):
            return Point()

        if P.x == Q.x and P.y == Q.y:
            slope = (3 * P.x**2 + self.a) * pow(2 * P.y, -1, self.p) % self.p
        else:
            slope = (Q.y - P.y) * pow(Q.x - P.x, -1, self.p) % self.p

        R = Point()
        R.x = (slope**2 - P.x - Q.x) % self.p
        R.y = (slope * (P.x - R.x) - P.y) % self.p
        return R

    def calc_point_subtraction(self, P, Q):
        if Q.is_infinity():
            return P
        if P.is_infinity():
            return Point(Q.x, (-Q.y) % self.p)

        Q_neg = Point(Q.x, (-Q.y) % self.p)
        return self.calc_point_add(P, Q_neg)

    def calc_point_multiplication(self, P, k):
        R = Point()
        current_point = P
        while k > 0:
            if k % 2 == 1:
                R = self.calc_point_add(R, current_point)
            current_point = self.calc_point_add(current_point, current_point)
            k //= 2
        return R

    def generate_keys(self):
        private_key = random.randint(1, self.p - 1)
        public_key = self.calc_point_multiplication(self.base_point, private_key)
        return private_key, public_key

    def encrypt(self, plaintext_point, public_key, k=None):
        if k is None:
            k = random.randint(1, self.p - 1)

        C1 = self.calc_point_multiplication(self.base_point, k)
        k_e2 = self.calc_point_multiplication(public_key, k)
        C2 = self.calc_point_add(plaintext_point, k_e2)

        return C1, C2

    def decrypt(self, C1, C2, private_key):
        d_C1 = self.calc_point_multiplication(C1, private_key)
        plaintext_point = self.calc_point_subtraction(C2, d_C1)
        return plaintext_point

    def get_all_points(self):
        points = [Point()]
        for x in range(self.p):
            y_squared = self.elliptic_curve_equation(x)
            for y in range(self.p):
                if (y**2) % self.p == y_squared:
                    points.append(Point(x, y))
        return points

    def create_mappings(self):
        valid_points = [point for point in self.valid_points]
        if len(valid_points) < len(self.characters):
            raise ValueError("Mismatch between the number of valid points and characters.")

        point_to_char = {point: char for point, char in zip(valid_points, self.characters)}
        char_to_point = {char: point for point, char in point_to_char.items()}
        return point_to_char, char_to_point

    def encode_character(self, char):
        return self.char_to_point[char]

    def decode_point(self, point):
        return self.point_to_char[point]

    def encrypt_message(self, message, public_key):
        ciphertext = ""
        for char in message:
            plaintext_point = self.encode_character(char)
            C1, C2 = self.encrypt(plaintext_point, public_key)

            # Menggabungkan x dan y dari C2 ke dalam satu nilai unik
            combined_value = (C2.x * self.p + C2.y) % 65536  # Memastikan tetap dalam rentang karakter Unicode
            encrypted_char = chr(combined_value)

            ciphertext += encrypted_char

        return ciphertext

    def decrypt_message(self, ciphertext, private_key):
        plaintext = ""
        for char in ciphertext:
            combined_value = ord(char)
            x = combined_value // self.p
            y = combined_value % self.p
            C2 = Point(x, y)

            C1 = self.base_point  # Menggunakan base_point untuk menghasilkan kembali C1
            decrypted_point = self.decrypt(C1, C2, private_key)
            plaintext += self.decode_point(decrypted_point)

        return plaintext

# Contoh Penggunaan
ecc = EllipticCurveElGamal()
private_key, public_key = ecc.generate_keys()

plaintext = "hello"
ciphertext = ecc.encrypt_message(plaintext, public_key)
decrypted_text = ecc.decrypt_message(ciphertext, private_key)

print(f"Plaintext: {plaintext}")
print(f"Ciphertext: {ciphertext}")  # Seharusnya 1:1 dengan plaintext
print(f"Decrypted: {decrypted_text}")  # Harus sama dengan plaintext


Plaintext: hello
Ciphertext: ࡙槿丯誁Შ
Decrypted: 	l¦7ç


In [31]:
import math

# Define elliptic curve parameters
p = 751  # Prime modulus
a = -1
b = 188
k = 20  # Scaling factor

def elliptic_curve_equation(x):
    """Compute the right-hand side of the elliptic curve equation."""
    return (x**3 - x + b) % p

def is_quadratic_residue(y_squared):
    """Check if y_squared is a quadratic residue modulo p."""
    return pow(y_squared, (p - 1) // 2, p) == 1

def find_valid_point(m):
    """Encode a character to a point on the elliptic curve."""
    for c in range(1, k + 1):  # Try different c values
        x = m * k + c
        y_squared = elliptic_curve_equation(x) % p

        if is_quadratic_residue(y_squared):  # Check if y^2 has a square root
            for y in range(p):
                if (y * y) % p == y_squared:
                    return x, y  # Return valid point

    raise ValueError(f"Could not find a valid point for m={m}")

def decode_point(x):
    """Decode a point to recover the original character index."""
    m = (x - 1) // k
    return m

# Example: Encoding and Decoding
char_to_num = {chr(i): i - 31 for i in range(32, 127)}  # ASCII printable characters
num_to_char = {v: k for k, v in char_to_num.items()}

# Encode a message
message = "HELLOSS@@@"
encoded_points = [find_valid_point(char_to_num[char]) for char in message]

# Decode back to message
decoded_message = "".join(num_to_char[decode_point(x)] for x, y in encoded_points)

# Display results
encoded_points, decoded_message


([(823, 285),
  (763, 235),
  (903, 337),
  (903, 337),
  (961, 97),
  (1042, 16),
  (1042, 16),
  (661, 179),
  (661, 179),
  (661, 179)],
 'HELLOSS@@@')

In [32]:
encoded_points2 = encoded_points
encoded_points2

[(823, 285),
 (763, 235),
 (903, 337),
 (903, 337),
 (961, 97),
 (1042, 16),
 (1042, 16),
 (661, 179),
 (661, 179),
 (661, 179)]

In [33]:
decoded_message2 = "".join(num_to_char[decode_point(x)] for x, y in encoded_points2)
decoded_message2

'HELLOSS@@@'

In [34]:
encoded_points_new = [find_valid_point(char_to_num[char]) for char in decoded_message2]
encoded_points_new

[(823, 285),
 (763, 235),
 (903, 337),
 (903, 337),
 (961, 97),
 (1042, 16),
 (1042, 16),
 (661, 179),
 (661, 179),
 (661, 179)]

In [35]:
import random

class Point:
    def __init__(self, x=None, y=None):
        self.x = x
        self.y = y

    def is_infinity(self):
        """Check if the point is the point at infinity."""
        return self.x is None and self.y is None

    def __eq__(self, other):
        """Custom equality check for Point objects."""
        if isinstance(other, Point):
            return self.x == other.x and self.y == other.y
        return False

    def __hash__(self):
        """Make Point hashable by defining a unique hash."""
        return hash((self.x, self.y))

    def __repr__(self):
        if self.is_infinity():
            return "Point at Infinity"
        return f"Point({self.x}, {self.y})"

class EllipticCurveElGamal:
  def __init__(self):
    self.a = 214
    self.b = 110
    # self.p = 251
    self.p = 233
    self.base_point = self.generate_random_valid_point()

    # self.characters = [chr(i) for i in range(256)]
    self.characters = [chr(i) for i in range(1, 256)]

    self.valid_points = self.get_all_points()
    self.point_to_char, self.char_to_point = self.create_mappings()

  def elliptic_curve_equation(self, x):
    return (x**3 + self.a*x + self.b) % self.p

  def is_on_curve(self, x, y):
        """Check if a point (x, y) lies on the curve."""
        if x is None or y is None:
            return True
        return (y**2 - (x**3 + self.a * x + self.b)) % self.p == 0

  def generate_random_valid_point(self):
        """Generate a random point that lies on the elliptic curve."""
        while True:
            x = random.randint(0, self.p - 1)
            y_squared = (x**3 + self.a * x + self.b) % self.p

            if pow(y_squared, (self.p - 1) // 2, self.p) == 1:
                for y in range(self.p):
                    if (y**2) % self.p == y_squared:
                        return Point(x, y)

  def calc_point_add(self, P, Q):
    """Calculate the addition of two points P and Q on the elliptic curve."""
    R = Point()

    if P.is_infinity():
        return Q
    if Q.is_infinity():
        return P

    if P.x == Q.x and (P.y != Q.y or P.y == 0):
        return Point()

    # Calculate slope
    if P.x == Q.x and P.y == Q.y:
        slope = (3 * P.x**2 + self.a) * pow(2 * P.y, -1, self.p) % self.p
    else:
        slope = (Q.y - P.y) * pow(Q.x - P.x, -1, self.p) % self.p

    R.x = (slope**2 - P.x - Q.x) % self.p

    R.y = (slope * (P.x - R.x) - P.y) % self.p

    return R

  def calc_point_doubling(self, P):
      """Calculate the point doubling 2P = P + P on the elliptic curve."""
      R = Point()

      if P.is_infinity() or P.y == 0:
          return Point()

      slope = (3 * P.x**2 + self.a) * pow(2 * P.y, -1, self.p) % self.p

      R.x = (slope**2 - 2 * P.x) % self.p

      R.y = (slope * (P.x - R.x) - P.y) % self.p

      return R

  def calc_point_subtraction(self, P, Q):
    """Calculate the subtraction of two points P - Q on the elliptic curve."""
    if Q.is_infinity():
        return P

    if P.is_infinity():
        Q_neg = Point(Q.x, (-Q.y) % self.p)
        return Q_neg

    Q_neg = Point(Q.x, (-Q.y) % self.p)

    return self.calc_point_add(P, Q_neg)


  def calc_point_multiplication(self, P, k):
    """Calculate kP using the double-and-add method."""
    R = Point()
    current_point = P

    while k > 0:
        if k % 2 == 1:
            R = self.calc_point_add(R, current_point)
        current_point = self.calc_point_add(current_point, current_point)
        k //= 2

    return R

  def generate_keys(self):
        """Generate a private and public key pair."""
        private_key = random.randint(1, self.p - 1)

        public_key = self.calc_point_multiplication(self.base_point, private_key)

        return private_key, public_key

  def encrypt(self, plaintext_point, public_key, k=None):
      """
      Encrypt a point on the elliptic curve using the public key.
      """
      if k is None:
          k = random.randint(1, self.p - 1)

      C1 = self.calc_point_multiplication(self.base_point, k)

      k_e2 = self.calc_point_multiplication(public_key, k)

      C2 = self.calc_point_add(plaintext_point, k_e2)

      return C1, C2


  def decrypt(self, C1, C2, private_key):
        """
        Decrypt a ciphertext pair (C1, C2) using the private key.
        """
        d_C1 = self.calc_point_multiplication(C1, private_key)

        plaintext_point = self.calc_point_subtraction(C2, d_C1)

        return plaintext_point

  def get_all_points(self):
      """
      Generate all valid points on the elliptic curve.
      """
      points = [Point()]
      for x in range(self.p):
          y_squared = self.elliptic_curve_equation(x)
          for y in range(self.p):
              if (y**2) % self.p == y_squared:
                  points.append(Point(x, y))

      return points

  def create_mappings(self):
    valid_points = [point for point in self.valid_points]

    if len(valid_points) != len(self.characters):
        raise ValueError("Mismatch between the number of valid points and characters.")

    point_to_char = {point: char for point, char in zip(valid_points, self.characters)}

    char_to_point = {char: point for point, char in point_to_char.items()}

    return point_to_char, char_to_point


  def encode_character(self, char):
        """Encode a character to a point on the elliptic curve."""
        if char not in self.char_to_point:
            raise ValueError(f"Character '{char}' not in mapping.")

        return self.char_to_point[char]

  def decode_point(self, point):
        """Decode a point on the elliptic curve to a character."""
        if point not in self.point_to_char:
            raise ValueError(f"Point '{point}' not in mapping.")

        return self.point_to_char[point]

  def encrypt_message(self, message, public_key):
      """
      Encrypt a message using the elliptic curve encryption scheme and return a character-based ciphertext.
      """
      ciphertext = ""

      for char in message:
          plaintext_point = self.encode_character(char)

          C1, C2 = self.encrypt(plaintext_point, public_key)

          print(C1, C2)

          encrypted_char_C1 = self.decode_point(C1)
          encrypted_char_C2 = self.decode_point(C2)

          ciphertext += encrypted_char_C1 + encrypted_char_C2

      return ciphertext

  # def decrypt_message(self, ciphertext, private_key):
  #     """
  #     Decrypt a ciphertext into its plaintext message using the private key.
  #     """
  #     plaintext = ""
  #     for i in range(0, len(ciphertext), 2):
  #         C1 = self.encode_character(ciphertext[i])
  #         C2 = self.encode_character(ciphertext[i + 1])

  #         decrypted_point = self.decrypt(C1, C2, private_key)
  #         char = self.decode_point(decrypted_point)
  #         plaintext += char

  #     return plaintext

  def decrypt_message(self, ciphertext, private_key):
    """
    Decrypt a ciphertext into its plaintext message using the private key.
    """
    if len(ciphertext) % 2 != 0:
        raise ValueError("Ciphertext length must be even to form valid (C1, C2) pairs.")

    plaintext = ""

    for i in range(0, len(ciphertext), 2):
        C1 = self.encode_character(ciphertext[i])
        C2 = self.encode_character(ciphertext[i + 1])

        decrypted_point = self.decrypt(C1, C2, private_key)
        char = self.decode_point(decrypted_point)
        plaintext += char

    return plaintext


In [36]:
eceg = EllipticCurveElGamal()
private_key, public_key = eceg.generate_keys()

message = "Hello, this is a secret message!"

ciphertext = eceg.encrypt_message(message, public_key)

decrypted_message = eceg.decrypt_message(ciphertext, private_key)

assert message == decrypted_message, "Decrypted message does not match the original!"

message, decrypted_message

Point(68, 104) Point(79, 22)
Point(154, 40) Point(204, 182)
Point(162, 182) Point(121, 73)
Point(62, 149) Point(85, 210)
Point(143, 53) Point(155, 206)
Point(62, 84) Point(210, 206)
Point(150, 135) Point(193, 53)
Point(131, 161) Point(169, 204)
Point(129, 226) Point(209, 217)
Point(77, 197) Point(149, 194)
Point(13, 14) Point(193, 180)
Point(140, 111) Point(157, 102)
Point(173, 141) Point(114, 95)
Point(14, 228) Point(157, 131)
Point(81, 183) Point(155, 27)
Point(168, 31) Point(207, 152)
Point(68, 129) Point(36, 114)
Point(81, 50) Point(206, 38)
Point(141, 15) Point(192, 231)
Point(122, 30) Point(62, 84)
Point(224, 22) Point(179, 134)
Point(90, 48) Point(104, 138)
Point(217, 38) Point(219, 121)
Point(167, 218) Point(179, 134)
Point(150, 98) Point(211, 150)
Point(227, 63) Point(161, 104)
Point(129, 7) Point(114, 138)
Point(227, 63) Point(106, 87)
Point(150, 98) Point(149, 39)
Point(39, 111) Point(88, 4)
Point(77, 36) Point(95, 77)
Point(212, 175) Point(114, 138)


('Hello, this is a secret message!', 'Hello, this is a secret message!')

In [37]:
ciphertext

'>H¢Ý±v9O\x92¥8é\x9dÐ\x87»\x83çG\x9b\x0cÑ\x8c¨¿n\x0f©K¤¸ã? Jà\x8eÏx8øÃTeîó·Ã\x9cëü®\x82oüh\x9c\x9a$RFXío'

In [38]:
import requests
from PIL import Image
import io
import numpy as np

# URL to the raw image file
url = "https://raw.githubusercontent.com/mikolalysenko/lena/master/lena.png"

# Download the image
response = requests.get(url)
if response.status_code == 200:
    # Load the image using PIL
    lena_image = Image.open(io.BytesIO(response.content))
    lena_image.show()  # Display the image (optional)
    lena_image.save("lena.png")  # Save the image locally
else:
    print("Failed to download the image.")

In [39]:
import os
LENA_IMG = 'lena.png'

if not os.path.exists(LENA_IMG):
    raise FileNotFoundError(f"Image not found at {LENA_IMG}")

In [40]:
from PIL import Image
import numpy as np

class ImprovedLSB_XOR:
    def __init__(self):
        """Initialize the improved LSB steganography using XOR/XNOR."""
        pass

    @staticmethod
    def xor_operation(bit1, bit2):
        """Perform XOR operation on two bits."""
        return str(int(bit1) ^ int(bit2))

    @staticmethod
    def xnor_operation(bit1, bit2):
        """Perform XNOR operation on two bits."""
        return str(int(not (int(bit1) ^ int(bit2))))

    def embed_message(self, input_image_path, output_image_path, message):
        """
        Embed a message in an image using an improved LSB technique with XOR/XNOR.
        """
        # Convert message to binary
        message_bits = ''.join(format(ord(char), '08b') for char in message)
        message_length = format(len(message_bits), '032b')  # Store length in 32-bit

        # Combine message length and message
        full_message_bits = message_length + message_bits

        # Load image
        image = Image.open(input_image_path)
        img_data = np.array(image)

        height, width, channels = img_data.shape
        bit_idx = 0

        for h in range(height):
            for w in range(width):
                if bit_idx >= len(full_message_bits):
                    break

                # Extract Red, Green, and Blue channels
                red_pixel = img_data[h, w, 0]  # R channel
                green_pixel = img_data[h, w, 1]  # G channel
                blue_pixel = img_data[h, w, 2]  # B channel

                # Get the most significant bit of the red channel
                red_msb = format(red_pixel, '08b')[0]

                # Determine whether to use XOR or XNOR
                if h % 2 == 0:
                    # Even row → XOR operation, embed in Green channel LSB
                    modified_bit = self.xor_operation(red_msb, full_message_bits[bit_idx])
                    modified_pixel = (green_pixel & 0b11111110) | int(modified_bit)
                    img_data[h, w, 1] = modified_pixel  # Modify green channel
                else:
                    # Odd row → XNOR operation, embed in Blue channel LSB
                    modified_bit = self.xnor_operation(red_msb, full_message_bits[bit_idx])
                    modified_pixel = (blue_pixel & 0b11111110) | int(modified_bit)
                    img_data[h, w, 2] = modified_pixel  # Modify blue channel

                bit_idx += 1

        # Save stego image
        stego_image = Image.fromarray(img_data)
        stego_image.save(output_image_path, format="PNG")
        print("Message embedded successfully.")

    def extract_message(self, stego_image_path):
        """
        Extract a message from an image using the improved XOR/XNOR-based LSB technique.
        """
        image = Image.open(stego_image_path)
        img_data = np.array(image)

        height, width, channels = img_data.shape
        extracted_bits = ""

        for h in range(height):
            for w in range(width):
                # Extract Red, Green, and Blue channels
                red_pixel = img_data[h, w, 0]  # R channel
                green_pixel = img_data[h, w, 1]  # G channel
                blue_pixel = img_data[h, w, 2]  # B channel

                # Get the most significant bit of the red channel
                red_msb = format(red_pixel, '08b')[0]

                # Determine whether to use XOR or XNOR for extraction
                if h % 2 == 0:
                    # Even row → XOR operation, extract from Green channel LSB
                    extracted_bit = self.xor_operation(red_msb, format(green_pixel, '08b')[-1])
                else:
                    # Odd row → XNOR operation, extract from Blue channel LSB
                    extracted_bit = self.xnor_operation(red_msb, format(blue_pixel, '08b')[-1])

                extracted_bits += extracted_bit

        # Extract the stored message length
        message_length_bits = extracted_bits[:32]
        message_length = int(message_length_bits, 2)
        message_bits = extracted_bits[32:32 + message_length]

        # Convert bits to text
        message = ''.join(chr(int(message_bits[i:i+8], 2)) for i in range(0, len(message_bits), 8))
        return message


# Example Usage
stego = ImprovedLSB_XOR()

# Step 1: Embed Message
input_image = "lena.png"   # Change to your input image path
output_image = "stego.png"  # Output stego image
secret_message = "HELLO XOR LSB"

stego.embed_message(input_image, output_image, secret_message)

# Step 2: Extract Message
retrieved_message = stego.extract_message(output_image)
print("Extracted Message:", retrieved_message)


Message embedded successfully.
Extracted Message: HELLO XOR LSB
